# Customer and Product Segmentation
## Introduction

Customer segmentation has been completed; the resulting output file is: rfm_customer_segments.csv

Prerequisite: Create a "final" folder in the root directory where the segmentation processing .py file is located.

The objective is to extract the segments into a CSV file along with the top 10 products that appear most frequently in the segmented customers' baskets.

To achieve this, I will perform data merges across the various input files.

Input:
- csv/products.csv  
- csv/orders.csv  
- csv/order_products_prior.csv  
- rfm_customer_segments.csv  

Output:

- final/products_segment.csv: Will contain two columns (segment, products). The products will be ranked in order of importance (purchase frequency).

Constraints:

- The rfm_customer_segment file contains over 200,000 rows.
- The order_products_prior file contains over 32,000,000 rows.
- The orders file contains over 3,000,000 rows.
- The products file contains over 49,000 rows.

Instead of loading all columns from every file, the script will only read the strictly necessary columns. This is crucial to optimize memory usage and prevent out-of-memory errors.  

ID Linkages (Merges):

- In the rfm_customer_segments and orders files, we use the shared column: user_id
- In the orders and order_products_prior files, we use the shared column: order_id
- In the order_products_prior and products files, we use the shared column: product_id
- Finally, in the products file, we retrieve the column: product_name

In [2]:
import pandas as pd
import gc # Garbage Collector : pour libérer la RAM

In [11]:
# --- CONFIGURATION DES CHEMINS ---
FILE_RFM = "csv/rfm_customer_segments.csv"
FILE_ORDERS = "csv/orders.csv"
FILE_ORDER_PRODUCTS = "csv/order_products_prior.csv"
FILE_PRODUCTS = "csv/products.csv"
FILE_OUTPUT = "final/products_segment.csv"

In [4]:
def load_data():
    """
    Étape 1 : Chargement ciblé des données.
    On ne lit que les colonnes nécessaires pour économiser la RAM.
    """
    print("1. Chargement des fichiers.")
    
    rfm = pd.read_csv(FILE_RFM, usecols=['user_id', 'segment'])
    orders = pd.read_csv(FILE_ORDERS, usecols=['order_id', 'user_id'])
    
    # Ce fichier est énorme (32M lignes), il faut préciser le type de données (int32).
    order_products = pd.read_csv(FILE_ORDER_PRODUCTS, usecols=['order_id', 'product_id'], dtype={'order_id': 'int32', 'product_id': 'int32'})
    
    products = pd.read_csv(FILE_PRODUCTS, usecols=['product_id', 'product_name'])
    
    return rfm, orders, order_products, products

In [5]:
def merge_datasets(rfm, orders, order_products, products):
    """
    Étape 2 : Jointures successives (Merges).
    On supprime les tableaux intermédiaires au fur et à mesure pour ne pas saturer la RAM.
    """
    print("2. Début des liaisons (Merges)...")
    
    # A. Liaison Orders <-> RFM (via user_id)
    print("   Liaison Clients et Commandes...")
    df = orders.merge(rfm, on='user_id', how='inner')
    
    # Nettoyage mémoire : on n'a plus besoin des DataFrames originaux rfm et orders
    del rfm
    del orders
    gc.collect() 

    # B. Liaison Résultat <-> Order_Products (via order_id)
    print("   Liaison Commandes et Produits achetés (Opération lourde)...")
    df = df.merge(order_products, on='order_id', how='inner')
    
    del order_products
    # On n'a plus besoin de order_id et user_id pour la suite, on les supprime du tableau !
    df.drop(columns=['order_id', 'user_id'], inplace=True) 
    gc.collect()

    # C. Liaison Résultat <-> Products (via product_id)
    print("   Récupération des noms de produits...")
    df = df.merge(products, on='product_id', how='inner')
    
    del products
    # On n'a plus besoin du product_id, on ne garde que segment et product_name
    df.drop(columns=['product_id'], inplace=True)
    gc.collect()
    
    return df

In [6]:
def get_top_10_products(df):
    """
    Étape 3 : Comptage et extraction du Top 10.
    """
    print("3. Comptage des produits par segment...")
    
    # On compte combien de fois chaque produit apparait dans chaque segment
    counts = df.groupby(['segment', 'product_name']).size().reset_index(name='purchase_count')
    
    # On trie : par segment (alphabétique) puis par nombre d'achats (décroissant)
    print("   Tri et extraction du Top 10...")
    counts = counts.sort_values(by=['segment', 'purchase_count'], ascending=[True, False])
    
    # On groupe par segment et on ne garde que les 10 premières lignes
    top_10_df = counts.groupby('segment').head(10)
    
    return top_10_df

In [9]:
def format_and_save(top_10_df):
    """
    Étape 4 : Formatage final et Sauvegarde.
    Transforme les 10 lignes par segment en 1 seule ligne avec une liste séparée par des virgules.
    """
    print("4. Formatage du fichier final...")
    
    # Pour chaque segment, on prend la colonne product_name et on joint les valeurs avec ", "
    final_df = top_10_df.groupby('segment')['product_name'].apply(lambda x: ', '.join(x)).reset_index()
    
    # On renomme la colonne selon ta demande
    final_df.rename(columns={'product_name': 'products'}, inplace=True)
    
    # Sauvegarde
    print(f"   Sauvegarde dans {FILE_OUTPUT}...")
    final_df.to_csv(FILE_OUTPUT, index=False)
    
    print("-" * 30)
    print("SUCCÈS ! Traitement terminé.")

In [12]:
def main():
    # Exécution de la pipeline
    rfm, orders, order_products, products = load_data()
    df_merged = merge_datasets(rfm, orders, order_products, products)
    df_top_10 = get_top_10_products(df_merged)
    format_and_save(df_top_10)

if __name__ == "__main__":
    main()

1. Chargement des fichiers.
2. Début des liaisons (Merges)...
   Liaison Clients et Commandes...
   Liaison Commandes et Produits achetés (Opération lourde)...
   Récupération des noms de produits...
3. Comptage des produits par segment...
   Tri et extraction du Top 10...
4. Formatage du fichier final...
   Sauvegarde dans final/products_segment.csv...
------------------------------
SUCCÈS ! Traitement terminé.
